Sedaj, ko imamo podatke o študentih, nas zanima še kaj o predmetih, predvsem pass rate čez leta za vsak predmet. Spisali bomo funkcijo, ki iz dane tabele "obrne" podatke iz `student_id X semester` (celica vsebuje vse predmete študenta v semestru) v `subject_id X semester` (celica vsebuje vse študente na predmetu v semestru). Druga funkcija bo naredila isto, le da bo vrnila samo št. študentov.

In [3]:
import pandas as pd
import ast

students = pd.read_csv("./checkpoint/students_no_empty_rows.csv")
subjects = pd.read_csv("./subjects_3.csv")
subjects_name = subjects.iloc[:, :2]

def transform_to_subject_centric(students, subjects_name):
    # slovar ki bo hranil pare subject_id : seznam vseh student_id, ki so imeli ta predmet (po letih in semestrih)
    subject_data = {}

    year_semester_columns = students.columns[1:]

    for index, student in students.iterrows():
        student_id = student['student_id']

        for column in year_semester_columns:
            if not pd.isna(student[column]):
                subjects = ast.literal_eval(student[column])

                for subject_id in subjects:
                    if (subject_id[:3] == '632') or (subject_id[:3] == '637'):
                        if subject_id not in subject_data:
                            subject_data[subject_id] = {col: [] for col in year_semester_columns}

                        subject_data[subject_id][column].append(student_id)

    # iz slovarja naredimo data frame, ki bo neurejen, zato ga še posortiramo in dodamo imena predmetov
    subject_centric_df = pd.DataFrame.from_dict(subject_data, orient='index')
    subject_centric_df = subject_centric_df.sort_index()
    subject_centric_df = subject_centric_df.reset_index().rename(columns={'index': 'subject_id'})
    
    # dodamo imena predmetov
    subject_centric_df = pd.merge(subject_centric_df, subjects_name, on='subject_id', how='left')

    columns_order = ['subject_id', 'subject_name'] + [col for col in subject_centric_df.columns[1:-1]]
    subject_centric_df = subject_centric_df[columns_order]
    subject_centric_df = subject_centric_df.set_index('subject_id')

    return subject_centric_df


def count_students_per_subject(subjects_centric_view):
    subjects_count = subjects_centric_view.copy()
    for index, row in subjects_centric_view.iterrows():
        for col in subjects_centric_view.columns[2:]:
            subjects_count.at[index, col] = len(ast.literal_eval(row[col])) if ((pd.notna(row[col])) and (row[col] != [])) else 0
    return subjects_count

In [ ]:
transform_to_subject_centric(students, subjects_name).to_csv("./checkpoint/subject_centric_view.csv", index=True)
subjects_centric = pd.read_csv("./checkpoint/subject_centric_view.csv")
# shranimo in preberemo csv, da funkcija count_students_per_subjects pravilno dela
count_students_per_subject(subjects_centric).to_csv("./checkpoint/subject_centric_count.csv", index=False)

Zdaj se lahko lotimo računanja prepustnosti po predmetih. Ker imamo podatke le o študentih, ki so bili vpisani leta 2019 ali kasneje, to pomeni, da za leto 2019 lahko preverimo prepustnost pri predmetih 1. letnika. Iz tabele `subjects_3.csv` bomo vzeli podatke iz vrstice `student_year`. Značke nam povejo, v katerem letniku programa se ta predmet izvaja. Spisali bomo funkcijo, ki bo vzela najvišjo številko, saj so nekateri predmeti hkrati obvezni in izbirni. Potem bomo izračunali prepustnost od leta `2019 + najvišji letnik izvajanja` naprej. 

In [28]:
import re

def extract_numbers_from_list(numbers_list):
    digits = [digit for s in numbers_list for digit in re.findall(r'\d+', s)]
    return ''.join(digits) if digits else None


def highest_number(numbers_list):
    if numbers_list is None:
        return 0
    else:
        numbers_string = str(numbers_list)
        digits = [int(digit) for digit in numbers_string]
        return str(max(digits))
    

def combine_semesters(row, year_prefix):
    col1 = f"{year_prefix}_1"
    col2 = f"{year_prefix}_2"

    list1 = ast.literal_eval(row[col1]) if pd.notna(row[col1]) else []
    list2 = ast.literal_eval(row[col2]) if pd.notna(row[col2]) else []

    combined_data = [*list1, *list2]
    
    return combined_data



In [61]:
subjects_years = pd.read_csv("./subjects_3.csv")
subjects_years = subjects_years.iloc[:, [0, 2, 3, 6]]

subjects_years['student_year'] = subjects_years['student_year'].apply(extract_numbers_from_list)
subjects_years['student_year'] = subjects_years['student_year'].apply(highest_number)
subjects_years['subject_semester'] = subjects_years['subject_semester'].apply(lambda x: str(x))
subjects_years.head(5)

,subject_id,subject_program,subject_semester,student_year
0,63202,UNI,1,1
1,63203,UNI,1,1
2,63204,UNI,1,1
3,63205,UNI,1,1
4,63207,UNI,2,1


In [ ]:
# mergamo podatke o letniku in semestru v subjects_centric_years tabelo
subjects_centric_years = pd.read_csv("./checkpoint/subject_centric_view.csv")
subjects_centric_years.head(10)
subject_centric_years = pd.merge(subjects_centric_years, subjects_years, on='subject_id', how='left')
subject_centric_years = subject_centric_years.set_index('subject_id')
subject_centric_years = subject_centric_years.dropna(subset=['subject_name'])

for year in range(2019, 2025):
    year_prefix = f"{str(year)[2:]}/{str(year+1)[2:]}"
    subject_centric_years[str(year)] = subject_centric_years.apply(lambda row: combine_semesters(row, year_prefix), axis=1)
    subject_centric_years.drop(columns=[f"{year_prefix}_1", f"{year_prefix}_2"], inplace=True)

subject_centric_years.to_csv("./checkpoint/subject_centric_years.csv")
subject_centric_years.head(5)

,subject_name,subject_program,subject_semester,student_year,2019,2020,2021,2022,2023,2024
subject_id,,,,,,,,,,
63202,Osnove matematične analize,UNI,1,1,"[63190045, 63190046, 63190050, 63190052, 63190...","[63190012, 63190026, 63190046, 63190059, 63190...","[63190009, 63190012, 63190091, 63190243, 63200...","[63190091, 63190106, 63190243, 63200034, 63200...","[63190243, 63210138, 63210156, 63210165, 63210...","[63190243, 63210167, 63210238, 63220003, 63220..."
63203,Diskretne strukture,UNI,1,1,"[63190045, 63190046, 63190050, 63190052, 63190...","[63190012, 63190026, 63190046, 63190091, 63190...","[63190009, 63190012, 63190243, 63200027, 63200...","[63200034, 63200345, 63200488, 63210021, 63210...","[63210085, 63210167, 63220007, 63220019, 63220...","[63210085, 63220003, 63230014, 63230032, 63230..."
63204,Osnove digitalnih vezij,UNI,1,1,"[63190001, 63190002, 63190003, 63190004, 63190...","[63190012, 63190026, 63190046, 63190050, 63190...","[63190009, 63190114, 63200011, 63200027, 63200...","[63210028, 63210085, 63210128, 63210186, 63210...","[63210085, 63220007, 63220019, 63220092, 63220...","[63220003, 63230070, 63230158, 63230174, 63230..."
63205,Fizika,UNI,1,1,"[63190045, 63190046, 63190050, 63190052, 63190...","[63190012, 63190026, 63190091, 63190103, 63190...","[63190009, 63190032, 63190067, 63190091, 63190...","[63200034, 63200128, 63200248, 63210114, 63210...","[63200034, 63210138, 63210243, 63210301, 63210...","[63210243, 63220003, 63220172, 63220246, 63220..."
63207,Linearna algebra,UNI,2,1,"[63190045, 63190046, 63190050, 63190052, 63190...","[63190012, 63190026, 63190046, 63190050, 63190...","[63190009, 63190091, 63190127, 63190243, 63190...","[63190106, 63190127, 63200034, 63200074, 63200...","[63190127, 63210085, 63210138, 63210156, 63210...","[63190127, 63210085, 63210200, 63210238, 63210..."


Zdaj spišemo funkcijo, ki bo vzela tabelo `subject_centric_view` in izračunala, koliko študentov je vsako leto opravilo predmet.

In [6]:
def pass_rate(subject_centric):
    columns_to_keep = ['subject_id', 'subject_name', 'subject_program', 'subject_semester', 'student_year']
    subject_pass_rate = subject_centric[columns_to_keep].copy()

    for year in range(2019, 2024):
        subject_pass_rate[f'{year}'] = ''

    for index, row in subject_centric.iterrows():
        offset = 3 if int(row['student_year']) == 0 else int(row['student_year'])
        year_start = 2018 + offset
        year_end = 2023

        for year in range(year_start, year_end + 1):

            first_year = row[str(year)]
            next_year = row[str(year+1)]

            first_year = ast.literal_eval(first_year) if pd.notna(first_year) and first_year != "" else []
            next_year = ast.literal_eval(next_year) if pd.notna(next_year) and next_year != "" else []

            total = len(first_year)

            if total <= 5:
                subject_pass_rate.at[index, f'{year}'] = f"{total} students"
                continue

            overlap = len(set(first_year).intersection(set(next_year)))
            pass_rate = ((total - overlap) / total) * 100

            pass_rate_value = round(pass_rate, 1)
            subject_pass_rate.at[index, f'{year}'] = f"{pass_rate_value}%"

    return subject_pass_rate


In [ ]:
subject_centric = pd.read_csv("./checkpoint/subject_centric_years.csv")

pass_rate = pass_rate(subject_centric)

# subject ids to remove
subjects_del = ['63284D', '63702DJ', '63766B', '63767', '63776', '63779', '63780']
pass_rate = pass_rate[~pass_rate['subject_id'].isin(subjects_del)]

# dipsem vss ma 100% za 2023, weird

pass_rate.to_csv("./pass_rate.csv", index=False)

Za zabavo bomo preverili, koliko ljudi se je vsako leto vpisalo in po enem letu pustilo študij. Prehodnost nekaterih predmetov v 1. letniku je sumljvo visoka, saj ne moremo upoštevati ljudi, ki se drugo leto ne pojavijo preprosto zato, ker ne študirajo več. Za vsako leto bomo izračunali delež študentov, ki je pustil študij po enem letu v primerjavi z vsemi vpisanimi študenti v tistem letu.

In [ ]:
import pandas as pd
import ast


# ali ima student samo dva semestra v letu vpisa?
def has_two_semesters(row):
    enrollment_year = row['enrollment_year']
    
    enrollment_columns = [col for col in students.columns if col.startswith(enrollment_year)]
    other_columns = [col for col in students.columns if not col.startswith(enrollment_year) and col not in ['student_id', 'enrollment_year']]

    enrollment_semesters_non_empty = sum(len(row[col]) > 0 if isinstance(row[col], list) else 0 for col in enrollment_columns)
    other_semesters_empty = all((len(row[col]) == 0 if isinstance(row[col], list) else True) for col in other_columns)

    return enrollment_semesters_non_empty == 2 and other_semesters_empty


In [61]:
students = pd.read_csv("./students_final.csv")

students = students.map(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
students['enrollment_year'] = students['student_id'].astype(str).str[2:4]

students['only_2semesters'] = students.apply(has_two_semesters, axis=1).astype(int)

students.sample(5)

,student_id,19/20_1,19/20_2,20/21_1,20/21_2,21/22_1,21/22_2,22/23_1,22/23_2,23/24_1,23/24_2,24/25_1,24/25_2,enrollment_year,only_2semesters
2177,63240140,[],[],[],[],[],[],[],[],[],[],"[63705, 63702, 63703, 63704, 63701]","[63710, 63708, 63706, 63707, 63709]",24,1
510,63200165,[],[],"[63202, 63277, 63205, 63204, 63203]","[63212, 63209, 63215, 63207, 63278]","[63203, 63218, 63208]","[63220, 63207, 63278]","[63213, 63279, 63283]","[63217, 63280, 63278, 63207, 63220, 63216]","[63270, 63213, 63214, 63269, 63255, 63283]","[63287, 63216, 63271, 63256, 63253, 63281]","[63214, 63283]",[63281],20,0
2122,63240075,[],[],[],[],[],[],[],[],[],[],"[63705, 63703, 63702, 63701, 63704]","[63710, 63709, 63708, 63707, 63706]",24,1
36,63190080,"[63704, 63701, 63705, 63702, 63703]","[63710, 63706, 63708, 63707, 63709]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,1
1743,63230121,[],[],[],[],[],[],[],[],"[63702, 63703, 63705, 63704, 63701]","[63709, 63707, 63708, 63710, 63706]","[63716, 63720, 63711, 63714, 63749B]","[63723, 63725, 63728, 63727, 63774]",23,0


In [68]:
def grouped(students):
    grouped = students.groupby('enrollment_year').agg(
        total_students=('student_id', 'count'),
        quit_after_one_year=('only_2semesters', 'sum')
    )

    # Calculate the percentage
    grouped['percentage'] = round(((grouped['quit_after_one_year'] / grouped['total_students']) * 100), 2)
    grouped['percentage'] = grouped['percentage'].apply(lambda x: f"{x}%")

    grouped = grouped.drop('24')

    return grouped

grouped = grouped(students)
grouped.to_csv("./checkpoint/quit_after_one_year.csv", index=False)
grouped.head(10)

,total_students,quit_after_one_year,percentage
enrollment_year,,,
19,388,85,21.91%
20,399,88,22.06%
21,440,91,20.68%
22,416,118,28.37%
23,412,103,25.0%


Za konec pa bomo spisali še funkcijo, ki za vsak predmet najde študente, ki so najdlje opravljali ta predmet in koliko let.

In [108]:
import pandas as pd
from collections import Counter

def find_most_frequent_students(data):
    results = []

    for index, row in data.iterrows():
        students = []
        for year in range(2019, 2025):
            if str(year) in data.columns:
                students.extend(row[str(year)])

        student_counts = Counter(students)

        if student_counts:
            max_count = max(student_counts.values())

            most_frequent_students = [
                student for student, count in student_counts.items() if count == max_count
            ]

            results.append({
                'subject_id': row['subject_id'],
                'subject_name': row['subject_name'],
                'most_frequent_students': ', '.join(map(str, most_frequent_students)), 
                'year_count': max_count,
                'student_count': len(most_frequent_students)
            })
            
        else:
            results.append({
                'subject_id': row['subject_id'],
                'subject_name': row['subject_name'],
                'most_frequent_students': '',
                'year_count': 0,
                'student_count': 0
            })

    results = pd.DataFrame(results)
    sorted_results = results.sort_values(by='year_count', ascending=False)

    return sorted_results

In [113]:
subjects_view = pd.read_csv("./subject_centric_years.csv")

for col in subjects_view.columns:
    if col not in ['subject_id', 'subject_name', 'subject_program', 'subject_semester', 'student_year']:
        subjects_view[col] = subjects_view[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

results_df = find_most_frequent_students(subjects_view)
results_df.to_csv("./checkpoint/ponavljalci.csv", index=False)
results_df.head(20)

,subject_id,subject_name,most_frequent_students,year_count,student_count
0,63202,Osnove matematične analize,63190243,6,1
43,63278,Programiranje 2,63190127,6,1
4,63207,Linearna algebra,63190127,6,1
10,63215,Osnove informacijskih sistemov,63190323,5,1
60,63710,Osnove verjetnosti in statistike,63200267,5,1
59,63709,Operacijski sistemi,63200267,5,1
15,63220,Principi programskih jezikov,63190095,5,1
14,63219,Matematično modeliranje,63190323,5,1
12,63217,Operacijski sistemi,63190323,5,1
11,63216,Teorija informacij in sistemov,63190323,5,1


Iz tabele, ki nam po predmetih prikaže študente, ki so jih najdlje opravljali, bomo naredili seznam študentov, ki se največkrat pojavijo v tej tabeli. Spisali bomo še pomožno funkcijo, ki v tabeli ponavljalcev najde vrstice, v katerih so omenjeni študentje, da ugotovimo katere predmete so opravljali maksimalno možno zaenkrat.  

In [115]:
import pandas as pd
from collections import Counter

ponavljalci = pd.read_csv("./checkpoint/ponavljalci.csv")
ponavljalci_filtered = ponavljalci[ponavljalci['year_count'] >= 3]

student_lists = results_df['most_frequent_students'].str.split(', ')
all_students = [student for sublist in student_lists for student in sublist]

student_counts = Counter(all_students)

most_frequent_students = pd.DataFrame(
    student_counts.items(), columns=['student_id', 'frequency']
).sort_values(by='frequency', ascending=False)

most_frequent_students.to_csv("./checkpoint/najpogostejsi_ponavljalci.csv", index=False)

print(most_frequent_students)

    student_id  frequency
24    63190229          7
167   63200155          6
2     63190323          6
21    63190190          6
74    63190124          6
..         ...        ...
220   63200112          1
218   63220521          1
216   63210262          1
215   63210116          1
481   63220323          1

[482 rows x 2 columns]


In [117]:
def find_student_in_frequent_students(ponavljalci, student_id):
    ponavljalci = ponavljalci[ponavljalci['most_frequent_students'].str.contains(str(student_id))]
    return ponavljalci

In [ ]:
ponavljalci = pd.read_csv("./checkpoint/ponavljalci.csv")

student_to_find = 63190229

print(find_student_in_frequent_students(ponavljalci, student_to_find))

   subject_id                         subject_name  \
23      63724               Testiranje in kakovost   
27      63741                  Odločitveni sistemi   
30      63739  Robotika in računalniško zaznavanje   
43      63717            Organizacija računalnikov   
64      63732        Tehnologija programske opreme   
72      63248            Ekonomika in podjetništvo   
91      63720                   Umetna inteligenca   

                               most_frequent_students  year_count  \
23                                           63190229           4   
27                                           63190229           3   
30                                           63190229           3   
43  63200086, 63190229, 63200204, 63210086, 632103...           3   
64  63190442, 63190165, 63200111, 63200192, 632002...           2   
72                       63190229, 63200155, 63200192           2   
91  63190436, 63190061, 63190100, 63190121, 631902...           2   

    student_co